In [ ]:
import pickle
import numpy as np
import xarray as xr
import torch
from tqdm.notebook import tqdm

In [ ]:
layers = [f"OL{i}" for i in range(1, 5)] + [f"OA{i}" for i in range(1, 5)]
layers += ["LANDMASK", "HGT_M", "SOILTEMP", "SNOALB", "VAR", "CON", "VAR_SSO"]
data_train = {}
data_test = {}
for sn in tqdm([100, 120, 140, 160, 180]):
    for we in [100, 120, 140, 160, 180]:
        features = []
        #первые 160 файлов в каждой категории относим в train
        for idx in range(160):
            xr_data = xr.open_dataset(f'./geogrid_runs/geo_em.{we}x{sn}.{idx}.nc')
            domain = np.vstack([xr_data[layer].to_dataframe()[layer].values for layer in layers])
            features.append(domain.reshape(1, len(layers), sn - 1, we - 1))
        features = np.vstack(features)
        data_train[f'{we}x{sn}'] = torch.tensor(features)

        features = []
        #оставшиеся 40 файлов в каждой категории относим в test
        for idx in range(160, 200):
            xr_data = xr.open_dataset(f'./geogrid_runs/geo_em.{we}x{sn}.{idx}.nc')
            domain = np.vstack([xr_data[layer].to_dataframe()[layer].values for layer in layers])
            features.append(domain.reshape(1, len(layers), sn - 1, we - 1))
        features = np.vstack(features)
        data_test[f'{we}x{sn}'] = torch.tensor(features)

#сохранить на диск трейн и тест тензоры
with open('data_train_dict_torch.pkl', 'wb') as f:
    pickle.dump(data_train, f)

with open('data_test_dict_torch.pkl', 'wb') as f:
    pickle.dump(data_test, f)